# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.

## Imports

In [82]:
import pandas as pd
import altair as alt
from psycopg2 import connect
from psycopg2.extras import RealDictCursor
from dotenv import dotenv_values

## Setup

In [83]:
config = dotenv_values()
conn = connect(
    host=config["AWS_RDS_HOST"],
    port=config["AWS_RDS_PORT"],
    dbname=config["AWS_RDS_DBNAME"],
    user=config["AWS_RDS_USER"],
    password=config["AWS_RDS_PASSWORD"],
    cursor_factory=RealDictCursor
)

<h2>Points of interest</h2>
<ul>
    <li>How many emergencies where there?</li>
    <li>How many calls for assistance where there?</li>
    <li>Which exhibition has the most emergencies?</li>
    <li>Which exhibition has the most calls for assistance?</li>
    <li>Which floor has the most emergencies</li>
    <li>Which floor has the most calls for assistance?</li>
    <li>Which exhibition performs the best?</li>


</ul>

**Number of Emergencies**

In [84]:
with conn.cursor() as cursor:
    query = """
    SELECT COUNT(*) AS emergency_count
    FROM kiosk_transaction
    WHERE type = 'emergency';
    """
    cursor.execute(query)
    performance_result = cursor.fetchone()
    assistance_count = performance_result["emergency_count"] if performance_result else 0
    print(f"Emergency count: {assistance_count}")

Emergency count: 2


**Number of Assistance Requests**

In [85]:
with conn.cursor() as cursor:
    query = """
    SELECT COUNT(*) AS assistance_count
    FROM kiosk_transaction
    WHERE type = 'assistance';
    """
    cursor.execute(query)
    performance_result = cursor.fetchone()
    assistance_count = performance_result["assistance_count"] if performance_result else 0
    print(f"Assistance count: {assistance_count}")

Assistance count: 71


**Which exhibition has the most emergencies?**

In [86]:
with conn.cursor() as cursor:
    query = """
    SELECT e.name, count(*) AS emergency_count
    FROM kiosk_transaction kt
    JOIN exhibition e 
        ON kt.exhibition_id = e.exhibition_id
    WHERE type = 'emergency'
    GROUP BY e.name;
    """
    cursor.execute(query)
    assistance_df = pd.DataFrame(cursor.fetchall())
    print(assistance_df)

         name  emergency_count
0  Adaptation                2


**Which exhibition has the most calls for assistance?**

In [87]:
with conn.cursor() as cursor:
    query = """
    SELECT e.name, count(*) AS assistance_count
    FROM kiosk_transaction kt
    JOIN exhibition e 
        ON kt.exhibition_id = e.exhibition_id
    WHERE type = 'assistance'
    GROUP BY e.name
    ORDER BY assistance_count DESC;
    """
    cursor.execute(query)
    assistance_df = pd.DataFrame(cursor.fetchall())
    print(assistance_df)

                      name  assistance_count
0          Thunder Lizards                29
1               Adaptation                12
2  The Crenshaw Collection                11
3       Measureless to Man                10
4       Our Polluted World                 9


**Which floor has the most emergencies?**

In [88]:
with conn.cursor() as cursor:
    query = """
    SELECT e.floor, count(*) AS emergency_count
    FROM kiosk_transaction kt
    JOIN exhibition e
        ON kt.exhibition_id = e.exhibition_id
    WHERE type = 'emergency'
    GROUP BY e.floor
    ORDER BY emergency_count DESC;
    """
    cursor.execute(query)
    emergency_by_floor_df = pd.DataFrame(cursor.fetchall())
    print(emergency_by_floor_df)

   floor  emergency_count
0  Vault                2


**Which floor has the most calls for assistance?**

In [89]:
with conn.cursor() as cursor:
    query = """
    SELECT e.floor, count(*) AS assistance_count
    FROM kiosk_transaction kt
    JOIN exhibition e
        ON kt.exhibition_id = e.exhibition_id
    WHERE type = 'assistance'
    GROUP BY e.floor
    ORDER BY assistance_count DESC;
    """
    cursor.execute(query)
    assistance_by_floor_df = pd.DataFrame(cursor.fetchall())
    print(assistance_by_floor_df)

   floor  assistance_count
0      1                39
1  Vault                12
2      2                11
3      3                 9


In [90]:
assist_total = assistance_by_floor_df["assistance_count"].sum()
assist_chart_df = assistance_by_floor_df.assign(
    percentage=(
        assistance_by_floor_df["assistance_count"] / assist_total * 100).round(1)
)

assist_pie = (
    alt.Chart(assist_chart_df)
    .mark_arc(innerRadius=60, outerRadius=140)
    .encode(
        theta=alt.Theta("assistance_count:Q", stack=True),
        color=alt.Color(
            "floor:N",
            title="Floor",
            legend=alt.Legend(
                orient="top",          # Keep legend on right
                offset=0,                # Pull legend tight to chart
                padding=20,               # Remove extra legend padding
                titleFontSize=18,
                labelFontSize=16,
                symbolSize=180,
                titleLimit=300
            )
        ),
        tooltip=["floor", "assistance_count", "percentage"]
    )
    .properties(title=alt.TitleParams(
        text="Assistance Requests by Floor",
        offset=18,          # adds space between title and chart
        fontSize=22,
        anchor="start"
    ),
    padding={"top": 30, "left": 30, "right": 0, "bottom": 0},
    width=700, height=400
))

assist_pie.save('./analysis_images/assist_pie.png')
assist_pie

alt.Chart(...)

**Which exhibition performs the best?**

In [91]:
# Total Score where high scores means worse performing
with conn.cursor() as cursor:
    query = """
    SELECT e.name, SUM(value) as total_score
    FROM kiosk_transaction kt
    JOIN exhibition e
    ON kt.exhibition_id = e.exhibition_id
    GROUP BY e.name
    ORDER BY total_score DESC
    """
    cursor.execute(query)
    perf_result = pd.DataFrame(cursor.fetchall())
    print(perf_result)

                      name  total_score
0          Thunder Lizards         1336
1       Measureless to Man          820
2  The Crenshaw Collection          585
3       Our Polluted World          577
4               Adaptation          549


In [92]:
with conn.cursor() as cursor:
    query = """
    SELECT e.name, value, COUNT(value) as frequency
    FROM kiosk_transaction kt
    JOIN exhibition e
    ON kt.exhibition_id = e.exhibition_id
    GROUP BY e.name, value
    HAVING value is not null
    ORDER BY name, value
    """
    cursor.execute(query)
    incident_through_time = pd.DataFrame(cursor.fetchall())
    print(incident_through_time)

                       name  value  frequency
0                Adaptation      0        105
1                Adaptation      1        129
2                Adaptation      2         67
3                Adaptation      3         50
4                Adaptation      4         34
5        Measureless to Man      0         18
6        Measureless to Man      1        139
7        Measureless to Man      2        148
8        Measureless to Man      3         95
9        Measureless to Man      4         25
10       Our Polluted World      0         33
11       Our Polluted World      1         77
12       Our Polluted World      2         92
13       Our Polluted World      3         80
14       Our Polluted World      4         19
15  The Crenshaw Collection      0        166
16  The Crenshaw Collection      1        142
17  The Crenshaw Collection      2        113
18  The Crenshaw Collection      3         23
19  The Crenshaw Collection      4         37
20          Thunder Lizards      0

In [93]:
values = [0, 1, 2, 3, 4]
charts = []
height = 300
width = 900
for v in values:
    subset = (
        incident_through_time[incident_through_time["value"] == v]
        .sort_values("frequency", ascending=False)
        .head(5)
    )
    base = (
        alt.Chart(subset)
        .mark_bar()
        .encode(
            x=alt.X("name", axis=alt.Axis(labels=False, ticks=False, domain=False, title=None)),
            y=alt.Y("frequency", title="Frequency"),
            color=alt.Color("name", legend=None),
            tooltip=["name", "frequency"]
        )
        .properties(title=f"Value = {v}", height=height, width=width)
    )
    charts.append(base)

subset_v4 = (
    incident_through_time[incident_through_time["value"] == 4]
    .sort_values("frequency", ascending=False)
    .head(5)
)
final_chart = (
    alt.Chart(subset_v4)
    .mark_bar()
    .encode(
        x=alt.X("name", axis=alt.Axis(labels=False, ticks=False, domain=False, title=None)),
        y=alt.Y("frequency", title="Frequency"),
        color=alt.Color("name", legend=alt.Legend(title="Exhibition", orient="right",
                                                  labelFontSize=20,
                                                  titleFontSize=22,
                                                  symbolSize=400,
                                                  labelLimit=300
                                                  )),
        tooltip=["name", "frequency"]
    )
    .properties(title="Value = 4", height=height, width=width)
)
spacer = alt.Chart(pd.DataFrame({"s":[0]})).mark_text().encode().properties(width=60, height=320)

layout = (
    alt.hconcat(charts[0], charts[1]) &
    alt.hconcat(charts[2], charts[3]) &
    alt.hconcat(final_chart, spacer, spacer)
).resolve_scale(color='independent').properties(title=alt.Title("Exhibitions by Value", anchor="middle", fontSize=24))

layout.save('./analysis_images/exhibition_feedback_value.png')
layout

alt.VConcatChart(...)

**At what times to incidents occur?**

In [94]:
with conn.cursor() as cursor:
    query = """
    SELECT e.name, EXTRACT(HOUR FROM kt.transaction_time) AS hour, COUNT(*) AS incident_count
    FROM kiosk_transaction kt
    JOIN exhibition e
    ON kt.exhibition_id = e.exhibition_id
    WHERE kt.value is null
    GROUP BY e.name, hour
    ORDER BY e.name, hour
    """
    cursor.execute(query)
    incident_through_time = pd.DataFrame(cursor.fetchall())
    print(incident_through_time)

                       name hour  incident_count
0                Adaptation    9               2
1                Adaptation   10               2
2                Adaptation   11               1
3                Adaptation   12               1
4                Adaptation   13               1
5                Adaptation   14               2
6                Adaptation   16               2
7                Adaptation   17               1
8                Adaptation   18               2
9        Measureless to Man    8               1
10       Measureless to Man   10               1
11       Measureless to Man   13               1
12       Measureless to Man   14               3
13       Measureless to Man   15               1
14       Measureless to Man   16               1
15       Measureless to Man   17               2
16       Our Polluted World    8               1
17       Our Polluted World    9               2
18       Our Polluted World   12               1
19       Our Pollute

In [95]:
# Convert Decimal types to float to avoid JSON serialization errors
incident_through_time["incident_count"] = incident_through_time["incident_count"].astype(
    float)
incident_through_time["hour"] = incident_through_time["hour"].astype(float)

incident_heatmap = (
    alt.Chart(incident_through_time)
    .mark_rect()
    .encode(
        x=alt.X("hour:O",
                title="Hour of Day",
                axis=alt.Axis(labelAngle=0, format="d", tickMinStep=1)),
        y=alt.Y("name:N",
                title="Exhibition",
                sort=alt.EncodingSortField(field="incident_count", op="sum", order="descending")),
        color=alt.Color("incident_count:Q",
                        title="Number of Incidents",
                        # Changed to red color scheme
                        scale=alt.Scale(scheme="reds")),
        tooltip=["name", "hour", "incident_count"]
    )
    .properties(
        title=alt.TitleParams(
            text="Incidents Throughout the Day",
            fontSize=20,
            anchor="middle",
            offset=20
        ),
        width=700,
        height=500
    )
)

incident_text = (
    alt.Chart(incident_through_time)
    .mark_text(baseline="middle")
    .encode(
        x=alt.X("hour:O"),
        y=alt.Y("name:N"),
        text=alt.Text("incident_count:Q"),
        color=alt.condition(
            alt.datum.incident_count > 1,
            alt.value("white"),
            alt.value("black")
        )
    )
)
final_heatmap = incident_heatmap + incident_text

final_heatmap.save('./analysis_images/incident_heatmap.png')
final_heatmap

alt.LayerChart(...)

**Number of Recordings For Each Exhibition**

In [96]:
with conn.cursor() as cursor:
    query = """
    SELECT e.name, count(*) AS recording_count
    FROM kiosk_transaction kt
    JOIN exhibition e
        ON kt.exhibition_id = e.exhibition_id
    GROUP BY e.name
    ORDER BY recording_count DESC;
    """
    cursor.execute(query)
    frequency_df = pd.DataFrame(cursor.fetchall())
    print(frequency_df)

                      name  recording_count
0          Thunder Lizards              501
1  The Crenshaw Collection              492
2       Measureless to Man              435
3               Adaptation              399
4       Our Polluted World              310


In [97]:
recording_total = frequency_df["recording_count"].sum()
recording_chart_df = frequency_df.assign(
    percentage=(frequency_df["recording_count"] /
                recording_total * 100).round(1)
)

recording_pie = (
    alt.Chart(recording_chart_df)
    .mark_arc(innerRadius=60, outerRadius=140)
    .encode(
        theta=alt.Theta("recording_count:Q", stack=True),
        color=alt.Color(
            "name:N",
            title="Exhibition",
            legend=alt.Legend(
                orient="right",
                padding=10,
                titleFontSize=14,
                labelFontSize=12,
                symbolSize=100
            )
        ),
        tooltip=["name", "recording_count", "percentage"]
    )
    .properties(
        title=alt.TitleParams(
            text="Recordings Per Exhibition",
            fontSize=18,
            offset=20,
        ),
        padding=dict(top=30, left=0, right=40, bottom=30),
        width=500,
        height=400
    )
)
recording_pie.save('./analysis_images/recording_frequency_pie.png')
recording_pie

alt.Chart(...)

**Distribution of Values**

In [98]:
with conn.cursor() as cursor:
    query = """
    SELECT e.name, kt.value 
    FROM kiosk_transaction kt
    JOIN exhibition e
        ON kt.exhibition_id = e.exhibition_id
    WHERE kt.value IS NOT NULL;
    """
    cursor.execute(query)
    values_df = pd.DataFrame(cursor.fetchall())
    print(values_df)

                         name  value
0     The Crenshaw Collection      0
1          Measureless to Man      1
2          Measureless to Man      3
3          Our Polluted World      3
4     The Crenshaw Collection      1
...                       ...    ...
2059               Adaptation      0
2060  The Crenshaw Collection      2
2061       Measureless to Man      3
2062          Thunder Lizards      3
2063          Thunder Lizards      2

[2064 rows x 2 columns]


In [99]:
values_df["value"] = values_df["value"].astype(float)

value_hist = (
    alt.Chart(values_df)
    .mark_bar()
    .encode(
        x=alt.X('value:Q',
                bin=alt.Bin(maxbins=5),
                title='Rating Value',
                scale=alt.Scale(domain=[0, 5])),
        y=alt.Y('count()', title='Frequency'),
        color=alt.value('#FF9999')
    )
    .properties(
        title=alt.TitleParams(
            text="Distribution of Recordings",
            fontSize=20,
            anchor="middle",
            offset=20
        ),
        width=900,
        height=300,
        padding={'top': 30, 'bottom': 30, 'left': 40, 'right': 30}
    )
)
value_hist.save('./analysis_images/value_distribution.png')
value_hist

alt.Chart(...)

In [100]:
values_df["value"] = values_df["value"].astype(float)

exhibition_boxplot = (
    alt.Chart(values_df)
    .mark_boxplot(size=30, extent='min-max')
    .encode(
        x=alt.X('name:N',
                title='Exhibition',
                axis=alt.Axis(labelAngle=-45)),
        y=alt.Y('value:Q',
                title='Rating Value',
                scale=alt.Scale(domain=[0, 5])),
        color=alt.Color('name:N',
                        legend=None)
    )
    .properties(
        title=alt.TitleParams(
            text="Distribution of Feedback Ratings by Exhibition",
            fontSize=20,
            anchor="middle",
            offset=20
        ),
        width=900,
        height=300,
        padding={'top': 30, 'bottom': 50, 'left': 40, 'right': 30}
    )
)
exhibition_boxplot.save('./analysis_images/exhibition_rating_boxplots.png')
exhibition_boxplot

alt.Chart(...)

In [101]:
conn.close()